## Search for product

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4 as nc4
import xarray as xr
import fsspec
import numpy as np
import xarray as xr
import planetary_computer
import pystac_client
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, time
import json

In [28]:
# Initialize PySTAC client for data query
planetary_computer.set_subscription_key("c27669c4bdec434d804e2bd738cb16fc")
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [39]:
#The acceptable formats are dd mm YYYYY, YYYY mm dd, dd-mm-YYYY, YYYY-mm-dd, dd/mm/YYYY, YYYY/mm/dd
start_date = "2023 08 20"
end_date = "25 08 2023"

#Insert bbox or country name
#Region = "New Zealand"
#Region = "Australia"
Region = [174.563615, -36.893762, 174.860246, -36.717901] #Auckland NZ
#Region = [-180, -90, 180, 90]

Date

In [26]:
# Function to convert date format 
def convert_format_date(input_date):
    correct_formats = ["%d %m %Y", "%Y %m %d", "%d/%m/%Y", "%Y/%m/%d", "%d-%m-%Y", "%Y-%m-%d"]
    
    for format_str in correct_formats:
        try:
            date_obj = datetime.strptime(input_date, format_str)
            formatted_date = date_obj.strftime("%Y-%m-%d")
            return formatted_date
        except ValueError: # Raised if input format is not compatible with set standard 
            pass
    
    raise ValueError("Invalid data format")

# Convert user start date format
try:
    start_date = convert_format_date(start_date)
except ValueError:
    print("Invalid start date format. Please check the acceptable formats")
            
# Convert user end date format
try:
    end_date = convert_format_date(end_date)
except ValueError:
    print("Invalid end date format. Please check the acceptable formats")

date_period = start_date + "/" + end_date 
print(date_period)

2023-08-20/2023-08-25


In [40]:
# Get geopandas in-built naturalearth_lowres dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Function to search the product by bbox and country
def search_catalog(Region, date_period):
    search_params = {
        "collections": "sentinel-5p-l2-netcdf",
        "datetime": date_period,
        "query": {"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
    }

    #bbox input
    if isinstance(Region, list) and len(Region) == 4:
        #-180 to 180 for longitudes, -90 to 90 for latitudes
        if all(-180 <= coordinates <= 180 for coordinates in [Region[0], Region[2]]) and all(-90 <= coordinates <= 90 for coordinates in [Region[1],Region[3]]):
            search_params["bbox"] = Region
        else:
            raise ValueError("Invalid coordinates in bbox")
        
    else:
        # country input
        # Extract coordinates of specified country and load into a JSON object
        ROI = world[world["name"] == Region]
        gjson = json.loads(ROI.to_json())
        # Create a MultiPolygon GeoJSON structure
        coordinates = gjson["features"][0]["geometry"]["coordinates"]
        if gjson["features"][0]["geometry"]["type"] == "Polygon": 
            coordinates = [coordinates]  
            # Convert to MultiPolygon
        search_params["intersects"] = {
            "type": "MultiPolygon", 
            "coordinates": coordinates,
        }

    search = catalog.search(**search_params)
    item = search.item_collection()

    return item

# Use search_catalog function with a single variable "Image" for both bbox and country
result = search_catalog(Region=Region, date_period=date_period)

# Print the result
print(f"Number of items for input: {len(result)}")

C:\Users\jenni\AppData\Local\Temp\ipykernel_26720\766764763.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


Number of items for input: 8
